In [1]:
import numpy as np
import requests
import json
from datetime import datetime
from collections import deque
import inspect

# Block-driven architecture 

In [2]:
def get_blocks_heights_in_last_n_hours(n: int):
    
    response = requests.get(url='https://api.bitaps.com/btc/v1/blockchain/blocks/last/'+str(n)+'/hours')
    json_data = json.loads(response.text)
    data = json_data['data']
    
    blocks_heights = [element['height'] for element in data]
    
    return blocks_heights

In [3]:
def initialize_metrics(size_in_blocks):
    
    fee_queue = deque([0]*size_in_blocks, size_in_blocks)
    block_pow_queue = deque([600]*size_in_blocks, size_in_blocks)
    cval_queue = deque([0]*size_in_blocks, size_in_blocks)
    txs_queue = deque([0]*size_in_blocks, size_in_blocks)
    unsigned_txs_queue = deque([0]*size_in_blocks, size_in_blocks)
    
    queues = {
        'fee':fee_queue,
        'block_pow':block_pow_queue,
        'cval':cval_queue,
        'txs_count':txs_queue,
        'unsigned_txs':unsigned_txs_queue
    } 
    
    metrics = {
        'fee':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0},
        'block_pow':{'tau':600, 'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':600*size_in_blocks},
        'cval':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0},
        'txs_count':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0},
        'unsigned_txs':{'avg':0, 'min':np.inf, 'max':-np.inf, 'sum':0}
    }
    
    metrics_dict = {
        'queues': queues,
        'metrics': metrics,
        'size':size_in_blocks,
    }
    
    return metrics_dict

In [4]:
heights = get_blocks_heights_in_last_n_hours(2)

In [5]:
metrics = initialize_metrics(5)

In [6]:
def update_metrics(old_metrics: dict, new_blocks_heights: list):
    
        
    def get_name(variable):

#         for fi in reversed(inspect.stack()):
#             names = [var_name for var_name, var_val in fi.frame.f_locals.items() if var_val is var]
#             if len(names) > 0:
#                 return names[0]
#         return (list(globals().keys()))[list(map(lambda x: id(x), list(globals().values()))).index(id(variable))]
        return(Retriever(variable))
            
    def get_unsigned_transaction_number():
    
        response = requests.get(url="https://blockchain.info/q/unconfirmedcount")
        count = json.loads(response.text)

        return count
    
    def get_block_info(block_height: int):
    
        url = 'https://api.blockchair.com/bitcoin/blocks?q=id(' + str(block_height) + ')'

        response = requests.get(url=url)
        json_data = json.loads(response.text)
        block = json_data['data'][0]

        # for Average interval between blocks pow
        time = datetime.strptime(block['time'],'%Y-%m-%d %H:%M:%S')
        # Miners fees
        fee = (block['fee_total'], block['fee_total_usd'])
        # Number of Transactions in a block
        txs_count = block['transaction_count']
        # Crypto-value in a block
        input_total = (block['input_total'],block['input_total_usd'])

        result = {
            'time':time,
#             'fee':{
#                 'satoshi':fee[0],
#                 'usd':fee[1]
#             },
            'fee':fee[0],
            'txs_count':txs_count,
#             'input_total':{
#                 'satoshi':input_total[0],
#                 'usd':input_total[1]
#             }
            'input_total':input_total[0]
        }

        return result


    def update_metrics_dict(new_metrics: list, metrics_dict:dict):

        name_map = {
            'fee':{'queues':'fee', 'metrics':'fee'},
            'time':{'queues':'block_pow', 'metrics':'block_pow'},
            'inpit_total':{'queues':'cval', 'metrics':'cval'},
            'txs_count':{'queues':'txs_count', 'metrics':'txs_count'},
            'unsigned_txs':{'queues':'unsigned_txs', 'metrics':'unsigned_txs'}
        }
        
        metrics_names_list = ['fee', 'time', 'inpit_total', 'txs_count', 'unsigned_txs']
        

        for idx, metric in enumerate(new_metrics):

            metric_name = metrics_names_list[idx]
            queue_name_map = name_map[metric_name]['queues']
            metric_name_map = name_map[metric_name]['metrics']
            metrics = metrics_dict['metrics'][metric_name_map]
            size = metrics_dict['size']

            poped = metrics_dict['queues'][queue_name_map].popleft()
            metrics_dict['queues'][queue_name_map].append(metric)
            metrics['sum'] -= poped
            metrics['sum'] += metric
            if metric < metrics['min']: 
                metrics['min'] = metric
            if metric > metrics['max']:
                metrics['max'] = metric
            metrics['avg'] = metrics['sum'] / size

            if 'tau' in metrics.keys():

                tau = metrics['tau']
                
                denominator = size / tau
                denominator += (1/metric - 1/poped)
                tau = size / denominator

                metrics['tau'] = tau

            metrics_dict['metrics'][metric_name_map] = metrics

        return metrics_dict


    for block_height in sorted(new_blocks_heights, reverse=False):
        
        print(block_height)
        
        block_info = get_block_info(block_height)
        
        fee = block_info['fee']
        time = (block_info['time'] - get_block_info(block_height-1)['time']).seconds
        inpit_total = block_info['input_total']
        txs_count = block_info['txs_count']
        unsigned_txs = get_unsigned_transaction_number()
        
        new_metrics = [fee, time, inpit_total, txs_count, unsigned_txs]
        
        updated_metrics_dict = update_metrics_dict(new_metrics, old_metrics)
        
    return updated_metrics_dict
        
    

In [7]:
updated = update_metrics(metrics, heights)

621856
621857
621858
621859
621860
621861
621862
621863


In [8]:
updated

{'queues': {'fee': deque([75121965, 75565393, 87938781, 68987478, 70550438]),
  'block_pow': deque([610, 637, 1906, 158, 377]),
  'cval': deque([1613734585061,
         1795215974118,
         3415373224189,
         1508623014111,
         1880613322437]),
  'txs_count': deque([2360, 1985, 2773, 2506, 1611]),
  'unsigned_txs': deque([22916, 22916, 22916, 22916, 22916])},
 'metrics': {'fee': {'avg': 75632811.0,
   'min': 68987478,
   'max': 96090110,
   'sum': 378164055},
  'block_pow': {'tau': 393.2210031407934,
   'avg': 737.6,
   'min': 158,
   'max': 2670,
   'sum': 3688},
  'cval': {'avg': 2042712023983.2,
   'min': 1508623014111,
   'max': 3582078633905,
   'sum': 10213560119916},
  'txs_count': {'avg': 2247.0, 'min': 1611, 'max': 3044, 'sum': 11235},
  'unsigned_txs': {'avg': 22916.0, 'min': 22916, 'max': 22916, 'sum': 114580}},
 'size': 5}